# Assess the Overlap of the CIViC coordinates with potential tumors


#### Usage

This script allows for tumors to be evaluated for overlap with the CIViC coordinates

#### Input Files:

1) file = text file that contains variant information. Each file requires a header with the following labels: sample, gene_name, VAF, amino_acid, chromosome_name, start, stop (these labels can be in any order). the sample column can be composed of multiple different tumors with distinct labels

#### Output Files:

1) mutation_overlap.tsv is a tab separated file that contains each tumor that is analyzed for overlap, the total number of overlaping variants with the smMIPs panel, and information on each variant that overlaps wih the smMIPs panel including: variant, gene, VAF.


In [1]:
##Tools
#!/usr/bin/env python3

#Create a tool that will append the input files to a list
def append_file_to_list(file):
    list = []
    f = open(file, 'r')
    for line in f:
        line = line.strip('\n')
        line = line.split('\t')
        list.append(line)
    return list

In [2]:
##evaluate the hodgkins lymphoma text file for 

def overlap(file, civic_coordinates):
    variant_list = append_file_to_list(file)
    mutation_overlap = {}
    count = 0
    header = variant_list[0]
    sample = header.index('sample')
    gene = header.index('gene_name')
    VAF = header.index('VAF')
    variant = header.index('amino_acid')
    chrom = header.index('chromosome_name')
    start = header.index('start')
    stop = header.index('stop')
    mutation_overlap = []
    for item in variant_list:
        for exon in civic_coordinates:
            if str(item[0]) == str(exon[0]) and int(item[1]) >= int(exon[1]) and int(item[2]) <= int(exon[2]):
                count += 1
                if item[sample] not in mutation_overlap:
                    mutation_overlap[item[sample]] = [0]
                    mutation_overlap[item[sample]][0] += 1
                    mutation_overlap[item[sample]].append(item[gene])
                    mutation_overlap[item[sample]].append(item[variant])
                    mutation_overlap[item[sample]].append(item[VAF])
                    mutation_overlap[item[sample]].append(item[chrom])
                    mutation_overlap[item[sample]].append(item[start])
                    mutation_overlap[item[sample]].append(item[stop])

                else:
                    mutation_overlap[item[sample]][0] += 1
                    mutation_overlap[item[sample]].append(item[gene])
                    mutation_overlap[item[sample]].append(item[variant])
                    mutation_overlap[item[sample]].append(item[VAF])
                    mutation_overlap[item[sample]].append(item[chrom])
                    mutation_overlap[item[sample]].append(item[start])
                    mutation_overlap[item[sample]].append(item[stop])
    return mutation_overlap

In [ ]:
#Pull in CIViC Information

In [ ]:
#Assess CIViC Coordinates with other info

In [ ]:
overlap = open('mutation_overlap.tsv', 'w')  #create empy file for nanostring coordinates
for k,v in mutation_overlap.items(): #iterate through nanostring list
    overlap.write(str(k) + '\t')
    for item in v:
        overlap.write(str(item) + '\t')
    overlap.write('\n')
overlap.close() #close file